In [1]:
import pandas as pd, numpy as np
import tpot
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('preprocessed.csv')

data.report_date = pd.to_datetime(data.report_date, format='%Y-%m-%d')
data['year'] = data['report_date'].dt.year
data['month'] = data['report_date'].dt.month
data['weekday'] = data['report_date'].dt.dayofweek
data['day'] = data['report_date'].dt.day
data['hour'] = data['report_date'].dt.hour

In [2]:
print(data.shape)
data.head()

(520, 88)


,report_date,city,online_store,offer_price,Chivalry_Juice,Chivalry_Sparkling,ChunYue_Water,Coke_Sparkling,Fanta_Sparkling,Georgia_Coffee,...,600.0x12,600.0x15,600.0x24,750.0x6,800.0x12,year,month,weekday,day,hour
0,2018-01-18,BeiJing,JD,7957.90,0.004717,0.009434,0.066038,0.250000,0.099057,0.023585,...,0.009434,0.028302,0.018868,0.000000,0.009434,2018,1,3,18,0
1,2018-01-18,BeiJing,YHD,3925.80,0.009174,0.018349,0.119266,0.229358,0.091743,0.018349,...,0.018349,0.045872,0.018349,0.000000,0.018349,2018,1,3,18,0
2,2018-01-18,ChangBei,Tmall_Supermarket,2136.36,0.000000,0.018182,0.018182,0.236364,0.090909,0.000000,...,0.000000,0.036364,0.018182,0.018182,0.000000,2018,1,3,18,0
3,2018-01-18,ChangChun,JD,6837.20,0.004717,0.009434,0.066038,0.250000,0.099057,0.018868,...,0.009434,0.028302,0.018868,0.000000,0.009434,2018,1,3,18,0
4,2018-01-18,ChangChun,YHD,4163.20,0.009174,0.018349,0.119266,0.229358,0.091743,0.018349,...,0.018349,0.045872,0.018349,0.000000,0.018349,2018,1,3,18,0


In [3]:
params = dict(
    generations=2, 
    population_size=100,
    offspring_size=30, 
    mutation_rate=0.7,
    crossover_rate=0.3,
    scoring='r2', 
    cv=5,
    n_jobs=3,
#     max_time_mins=15, 
    verbosity=2,
    disable_update_check=True
)

model = tpot.TPOTRegressor(**params)
model.fit(
    data.drop(['report_date', 'city', 'online_store', 'offer_price'], axis=1),
    data.offer_price
)

Optimization Progress:  81%|████████▏ | 130/160 [00:41<00:24,  1.25pipeline/s]

Generation 1 - Current best internal CV score: 0.990768454578987


Generation 2 - Current best internal CV score: 0.990768454578987

Best pipeline: KNeighborsRegressor(RobustScaler(input_matrix), n_neighbors=7, p=1, weights=distance)


TPOTRegressor(config_dict={'sklearn.linear_model.ElasticNetCV': {'l1_ratio': array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,
        0.9 ,  0.95,  1.  ]), 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}, 'sklearn.ensemble.ExtraT....45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ])}}}},
       crossover_rate=0.3, cv=5, disable_update_check=True,
       early_stop=None, generations=2, max_eval_time_mins=5,
       max_time_mins=None, memory=None, mutation_rate=0.7, n_jobs=3,
       offspring_size=30, periodic_checkpoint_folder=None,
       population_size=100, random_state=None, scoring=None, subsample=1.0,
       verbosity=2, warm_start=False)

In [23]:
import random
from deap import base
from deap import creator
from deap import tools
import multiprocessing

target_sale = 7000000
features = 84
population_size = 300
generation = 300
change_rate = (0.8, 0.4) #crossover rate/ mutation rate
tpot_model = model.fitted_pipeline_

def generator(n):
    return list(np.random.dirichlet(np.ones(n), size=1)[0])

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

# Attribute generator 
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", generator, features)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_bool)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ('fitness') function to be maximized
def model_prediction(individual):
    return tpot_model.predict(np.array(individual).reshape(1, -1))

def feasible(individual):
    if sum(individual[:24]) > 1 or sum(individual[24:]) > 1:
        return False
    return True

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", model_prediction)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 0))

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=3)

#----------

def main():

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=population_size)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = change_rate
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    # Begin the evolution
    while g < generation and max(fits) < target_sale:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print("  Evaluated %i individuals" % len(invalid_ind))
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    return (best_ind, best_ind.fitness.values)

%time best_param, best_sale = main()

Start of evolution
  Evaluated 300 individuals
-- Generation 1 --
  Evaluated 261 individuals
  Min 4105.3
  Max 7404.29688788
  Avg 6697.75751065
  Std 900.787927454
-- Generation 2 --
  Evaluated 261 individuals
  Min 0.0
  Max 7404.29692034
  Avg 6705.91476842
  Std 1264.73897191
-- Generation 3 --
  Evaluated 258 individuals
  Min 0.0
  Max 7404.29692115
  Avg 6805.6920447
  Std 1291.36067601
-- Generation 4 --
  Evaluated 265 individuals
  Min 0.0
  Max 7404.297981
  Avg 6531.46649687
  Std 2006.80638225
-- Generation 5 --
  Evaluated 270 individuals
  Min 0.0
  Max 7404.29914476
  Avg 6470.69936019
  Std 2287.16876055
-- Generation 6 --
  Evaluated 255 individuals
  Min 0.0
  Max 7404.30087559
  Avg 6204.39535258
  Std 2630.11402453
-- Generation 7 --
  Evaluated 262 individuals
  Min 0.0
  Max 7404.30266869
  Avg 6300.25810677
  Std 2532.72441427
-- Generation 8 --
  Evaluated 264 individuals
  Min 0.0
  Max 7404.30115659
  Avg 5987.36989537
  Std 2834.82686309
-- Generation 9 -

  Evaluated 253 individuals
  Min 0.0
  Max 7404.30771638
  Avg 3996.57584762
  Std 3559.981592
-- Generation 73 --
  Evaluated 260 individuals
  Min 0.0
  Max 7404.30771638
  Avg 3935.18942541
  Std 3596.79849804
-- Generation 74 --
  Evaluated 269 individuals
  Min 0.0
  Max 7404.30771638
  Avg 3911.91723144
  Std 3592.29578252
-- Generation 75 --
  Evaluated 263 individuals
  Min 0.0
  Max 7404.30771638
  Avg 3884.09104856
  Std 3541.30546798
-- Generation 76 --
  Evaluated 266 individuals
  Min 0.0
  Max 7404.30771628
  Avg 3426.46512651
  Std 3556.54613354
-- Generation 77 --
  Evaluated 269 individuals
  Min 0.0
  Max 7404.30771628
  Avg 3892.29633491
  Std 3566.46786306
-- Generation 78 --
  Evaluated 268 individuals
  Min 0.0
  Max 7404.30771582
  Avg 3962.91938011
  Std 3610.12154179
-- Generation 79 --
  Evaluated 270 individuals
  Min 0.0
  Max 7404.30771638
  Avg 3793.94254169
  Std 3634.24167863
-- Generation 80 --
  Evaluated 259 individuals
  Min 0.0
  Max 7404.30771645


  Evaluated 263 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4206.56463316
  Std 3642.02764624
-- Generation 143 --
  Evaluated 267 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4033.79745312
  Std 3661.80394504
-- Generation 144 --
  Evaluated 267 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3998.62391595
  Std 3654.59191642
-- Generation 145 --
  Evaluated 266 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4389.8243247
  Std 3622.25951181
-- Generation 146 --
  Evaluated 267 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4098.66932773
  Std 3639.14713618
-- Generation 147 --
  Evaluated 263 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4085.81703472
  Std 3625.47593277
-- Generation 148 --
  Evaluated 257 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3944.82513751
  Std 3632.48596756
-- Generation 149 --
  Evaluated 248 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4289.02327641
  Std 3618.51215284
-- Generation 150 --
  Evaluated 266 individuals
  Min 0.0
  Max 7404.

  Evaluated 267 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4174.78935032
  Std 3651.41627526
-- Generation 213 --
  Evaluated 261 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4075.02683198
  Std 3652.31250716
-- Generation 214 --
  Evaluated 267 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4114.93478744
  Std 3648.42130145
-- Generation 215 --
  Evaluated 264 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4149.06990918
  Std 3643.99326949
-- Generation 216 --
  Evaluated 274 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3953.27093183
  Std 3639.67958285
-- Generation 217 --
  Evaluated 261 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3848.99516522
  Std 3657.08685924
-- Generation 218 --
  Evaluated 266 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3881.8088635
  Std 3661.13991071
-- Generation 219 --
  Evaluated 259 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3882.31304446
  Std 3657.05478774
-- Generation 220 --
  Evaluated 257 individuals
  Min 0.0
  Max 7404.

  Evaluated 271 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3951.62164609
  Std 3662.81129497
-- Generation 283 --
  Evaluated 269 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4274.83487958
  Std 3631.10750235
-- Generation 284 --
  Evaluated 263 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4198.43196063
  Std 3637.59933883
-- Generation 285 --
  Evaluated 264 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3925.90226223
  Std 3653.84090201
-- Generation 286 --
  Evaluated 275 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4131.48314058
  Std 3640.97071593
-- Generation 287 --
  Evaluated 257 individuals
  Min 0.0
  Max 7404.30771742
  Avg 4035.11881929
  Std 3655.76392986
-- Generation 288 --
  Evaluated 266 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3984.43540167
  Std 3665.93954343
-- Generation 289 --
  Evaluated 264 individuals
  Min 0.0
  Max 7404.30771742
  Avg 3776.4946844
  Std 3665.84369713
-- Generation 290 --
  Evaluated 259 individuals
  Min 0.0
  Max 7404.

Process ForkPoolWorker-61:
Process ForkPoolWorker-60:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/opt/conda/l

In [4]:
target_sale = 7000000
percent_range = (0.0, 0.25)
features = 84
population_size = 300
generation = 2
change_rate = (0.8, 0.4) #crossover rate/ mutation rate

tpot_model = model.fitted_pipeline_

In [27]:
import numpy as np
import pandas as pd
from deap import creator, base, tools, algorithms
import multiprocessing

def generator(n):
    return list(np.random.dirichlet(np.ones(n), size=1)[0])

def model_prediction(individual):
    return tpot_model.predict(np.array(individual).reshape(1, -1))

def feasible(individual):
    if sum(individual[:24]) > 1 or sum(individual[24:]) > 1:
        return False
    return True

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)
toolbox.register("attr_bool", generator, features)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_bool)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", model_prediction)
# toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 0))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=population_size)
for gen in range(generation):
    offspring = algorithms.varAnd(pop, toolbox, cxpb=change_rate[0], mutpb=change_rate[1])
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    pop = toolbox.select(offspring, k=len(pop))
%time top = tools.selBest(pop, k=3)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 543 µs


In [28]:
top

[[0.0,
  0.017083850891396583,
  1.0,
  1.0,
  1.0,
  0.0,
  0.005781687420593597,
  0.0,
  1.0,
  0.0,
  0.055796258156948296,
  0.0041488429126322291,
  0.010493373956647487,
  0.0,
  0.0,
  0.0,
  0.0013315891801027606,
  0.0014087360569015682,
  0.0034518388223294923,
  1.0,
  0.0,
  0.018634551002829278,
  0.0,
  0.011618880462441609,
  0.018183910339580701,
  0.0,
  0.002598674043646929,
  0.00026398193718537411,
  0.0016383059853151492,
  0.0083671305595766667,
  0.0,
  1.0,
  0.00076599608215233476,
  0.0,
  0.021596589857496826,
  0.0,
  0.0,
  0.032280350049828689,
  0.0,
  0.0,
  0.008012131452042711,
  0.031489948508233306,
  0.0035255175207790268,
  0.0,
  0.0,
  0.0,
  0.030223392504814463,
  0.0,
  0.0,
  0.014373861712529483,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.011095868918085169,
  0.0,
  0.02155089386194942,
  0.0098875147435987763,
  0.0,
  0.012858341707964724,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0040542015261183315,
  0.0,
  0.0,
  0.0,
  0.0052499830384851393,
  0.